In [7]:
import numpy as np
import audio_metadata as am
import soundfile as sf                                                      
import scipy.io
import glob, os
import matplotlib.pyplot as plt
from IPython.display import Audio
%matplotlib inline

from Distortion import distortion, normalize
from Convolution import convolution

In [50]:
metadata = []
for filename in os.listdir("./Data"):
    if filename.endswith(('.wav', 'flac', 'mp3')):
        d = am.load(os.path.join('./Data', filename))['tags']
        metadata.append([d['title'][0], d['artist'][0], d['album'][0]])
        x, sr_d = sf.read(os.path.join('./Data', filename))
        for i in range(0, len(x), (10 * sr)):
            data = x[i : (i + (10 * sr_d))]
            if (len(np.shape(data)) > 1):
                data = np.mean(data, axis = 1)
            sf.write(os.path.join('./Data/Train', filename[:-5] + str(i // (10 * sr)) + '.wav'), data, sr_d)
            r = np.random.randint(0, 7)
            for irfilename in os.listdir("./Data/Impulses"):
                if (irfilename == "{}.wav".format(str(r))):
                    sr, ir = scipy.io.wavfile.read("./Data/Impulses/{}.wav".format(str(r)))
                    print(r, sr)
                if (len(np.shape(ir)) > 1):
                    ir = np.mean(ir, axis = 1)
                sf.write(os.path.join('./Data/Impulses{}.wav'.format(str(r))), ir, sr)
            convolution(os.path.join('./Data/Train', filename[:-5] + str(i // (10 * sr)) + '.wav'), os.path.join('./Data/Impulses', str(r) + '.wav'), os.path.join('./Data/Train', filename[:-5] + str(i // (10 * sr)) + '.wav'))
            sr_d, x = scipy.io.wavfile.read(os.path.join('./Data/Train', filename[:-5] + str(i) + '.wav'))
            p_td = np.random.random()
            p_cd = np.random.random()
            print(p_td, p_cd)
            if (p_td > 0.8):
                print("tanh distortion")
                x = distortion(x = x, gain = 2e-4, mode = 'tanh')
            if (p_cd > 0.8):
                print("clipping distortion")
                x = distortion(x = x, gain = 4, mode = 'clipping')
            scipy.io.wavfile.write(os.path.join('./Data/Train', filename[:-5] + str(i // (10 * sr)) + '.wav'), sr_d, x)
print(metadata)

    Ignoring ``TBPM``.
    Numeric text frame values must consist only of digits.
    


4 48000
0.3599328144382288 0.0899241914261032
3 48000


ValueError: max() arg is an empty sequence